In [1]:
import os
from BH.feature_functions import *

GPU_NUM = "0"
num_epochs = 500
batch_size = 128

use_ppath = False
column_info = "original"

graph_deg = 6
num_layers = graph_deg
num_features = 256
feature_list = {
    'constant':    (False, constant_feature),
    'column':      (False, column_indicator),
    'norm_column': (True, normalized_column_indicator), #   Feauture
    'norm_column_rev': (False, normalized_column_rev_indicator),
}
connected = False
UPTO = False

direction = "222"

shape_indicator = {
    'all_with_all_row_connectedness_criterion': (False, ),
    'all_with_inductive_connectedness_criterion': (False, ),
    '2row_less': (False, ),
    '2row_less_with_all_row_connectedness_criterion': (False, ),
    '2row_less_with_inductive_connectedness_criterion': (False, ),
    '3col_less_with_all_row_connectedness_criterion': (False, ),
    '3col_less_with_inductive_connectedness_criterion': (True, ),
}

shape= {
    'all': (False, ),# 3개다 
    '2row_less': (False, ),
    '3row_less': (True, ),
}
filter_indicator = {
    'with_all_row_connectedness_criterion':(False,), 
    'with_inductive_connectedness_criterion':(False,),
}


use_pretrained_weights = False
save_trained_weights = True

step_size = 0.0001
train_fraction = .8

DIR_PATH = f'/Data/Ptab/n={graph_deg}'
MODEL_DIR = './trained_models'
MODEL_FILE = os.path.join(MODEL_DIR, f'parameters_{graph_deg}_{num_layers}_{num_features}')

for key in shape_indicator:
    if shape_indicator[key][0] == True:
        DIR_PATH += f'_{key}'
        MODEL_FILE += f'_{key}'


if connected == True:
    DIR_PATH += "_connected"
    MODEL_FILE += "_connected"
elif connected == False:
    DIR_PATH += "_disconnected"
    MODEL_FILE += "_disconnected"
if UPTO == True:
    DIR_PATH += "_UPTO"
    MODEL_FILE += "_UPTO"
if not column_info == "original":
    DIR_PATH += column_info
    MODEL_FILE += column_info
if use_ppath:
    DIR_PATH += "_ppath"    
    MODEL_FILE += "_ppath"
    
for key in feature_list.keys():
    if feature_list[key][0] == True:
        MODEL_FILE += f'_{key}'
MODEL_FILE += f'_{direction}'
MODEL_FILE += '.pickle'



<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
print(DIR_PATH)

/Data/Ptab/n=6_3col_less_with_inductive_connectedness_criterion_disconnected


In [3]:
import functools
import enum
import os
from BH.data_loader import *
# from BH.generate_data import *
#from BH.data_loader import *
#from BH.generate_data import *
# from training_info import *

import pickle

from src.CustomDataset import *
from GCN_model import *

from torch_geometric.loader import DataLoader


os.environ["CUDA_VISIBLE_DEVICES"] = GPU_NUM
device = "cuda:" + GPU_NUM


print("Loading input data...")
full_dataset, train_dataset, test_dataset = load_input_data(DIR_PATH)

/root/anaconda3/envs/mpnn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading input data...
Generating data from the directory /Data/Ptab/n=6_3col_less_with_inductive_connectedness_criterion_disconnected


In [4]:
node_dim = num_features
edge_dim = 8
graph_deg = graph_deg
depth = num_layers

test_dataset = CustomDataset(test_dataset)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

train_dataset = CustomDataset(train_dataset)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = pastGCN().to(device)
if use_pretrained_weights == True:
    try:
        with open(MODEL_FILE, 'rb') as f:
          model, max_accuracy, min_loss = pickle.load(f)
          model.to(device)
    except:
        print("There is no trained_models model")
        use_pretrained_weights = False
if use_pretrained_weights == False:
    model = GCN_multi_conv(graph_deg, depth, node_dim, direction).to(device)
    max_accuracy = 0
    min_loss = 100

# data = batch.to(device)
# torch.nn.init.xavier_normal(model)
# loss_function = torch.nn.CrossEntropyLoss()
loss_function = torch.nn.NLLLoss()
# loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=step_size, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                        lr_lambda=lambda epoch: 0.993 ** epoch,
                                        last_epoch=-1,
                                        verbose=False)

In [5]:
for data in test_loader:
    print(data)
    break
data

DataBatch(x=[792, 1], edge_index=[2, 2772], y=[128], edge_types=[2772, 1], graph_sizes=[128], batch=[792], ptr=[129])


/root/Min/Ptab/CustomDataset.py:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484809662/work/torch/csrc/utils/tensor_new.cpp:201.)
  edge_index = torch.tensor([self.rows[idx], self.cols[idx]], dtype=torch.long)


DataBatch(x=[792, 1], edge_index=[2, 2772], y=[128], edge_types=[2772, 1], graph_sizes=[128], batch=[792], ptr=[129])

In [6]:
batch = data.batch
batch_size = len(data.y)
row_counts = torch.bincount(batch, minlength=batch_size)
num_graphs_per_batch = torch.div(row_counts, data.graph_sizes, rounding_mode='floor')
num_graphs = int(torch.sum(row_counts/data.graph_sizes))
max_columns = data.graph_sizes.max().item()
col_indices = torch.zeros_like(batch)    
graph_offset = torch.cumsum(num_graphs_per_batch, dim=0) - num_graphs_per_batch



In [7]:
xx = torch.full((num_graphs, max_columns, node_dim), 0.)

In [8]:
row_index = torch.cat([torch.arange(n_g).repeat_interleave(g_s) + offset
                       for n_g, g_s, offset in zip(num_graphs_per_batch, data.graph_sizes, graph_offset)])
# Compute column indices for each batch
col_index = torch.cat([torch.arange(g_s).repeat(n_g) for n_g, g_s in zip(num_graphs_per_batch, data.graph_sizes)])

xx[row_index, col_index, :] = data.x.view(-1, node_dim)

In [9]:
for epoch in range(num_epochs):
    # Training phase
    model.train()
    for batch in train_loader:
        batch.to(device)
        optimizer.zero_grad()
        out = model(batch)
        log_probs = torch.log(torch.clamp(out, min=1e-9))
#         log_probs = torch.log(out)
        loss = loss_function(log_probs, batch.y)
        loss.backward()
        # Apply gradient clipping
    
        optimizer.step()
    scheduler.step()
    print(loss)
    
    # Evaluation phase
    model.eval()
    correct = 0
    total = 0
#     with torch.no_grad():
#         for batch in test_loader:
#             batch.to(device)
#             outputs = model(batch)
# #             _,predicted = torch.max(outputs.data, 1)
#             predicted = outputs
#             total += batch.y.size(0)
# #             correct += (predicted == batch.y).sum().item()
#             correct += ((predicted - batch.y)**2<0.1).sum().item()
    with torch.no_grad():  # Disable gradient calculation
        total = 0
        correct = 0
        for batch in test_loader:
            # Move batch to the appropriate device (GPU or CPU)
            batch.to(device)

            # Forward pass: get model predictions
            outputs = model(batch)

            # Predicted classes are the indices of the max logits along the class dimension
            _, predicted = torch.max(outputs.data, 1)  # Shape: (batch_size)

            # Increment the total number of samples
            total += batch.y.size(0)

            # Compare predicted labels with ground truth labels
            correct += (predicted == batch.y).sum().item()  # Count the number of correct predictions

    # Compute the accuracy as a percentage
    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')
    # Compute accuracy
    accuracy = correct / total
    loss = float(loss.item())

    print("Epoch [{}/{}], Accuracy: {:.2%}, Loss: {:.15f}".format(epoch + 1, num_epochs, accuracy, loss))

    if (accuracy > max_accuracy or (accuracy == max_accuracy and loss < min_loss)) and save_trained_weights:
        max_accuracy = accuracy
        min_loss = loss
        with open(MODEL_FILE, 'wb') as f:
            pickle.dump((model, max_accuracy, min_loss), f)

tensor(1.0319, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 61.33%
Epoch [1/500], Accuracy: 61.33%, Loss: 1.031880259513855
tensor(0.8653, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 74.03%
Epoch [2/500], Accuracy: 74.03%, Loss: 0.865263223648071
tensor(0.3920, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 77.90%
Epoch [3/500], Accuracy: 77.90%, Loss: 0.391981184482574
tensor(0.4477, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 75.14%
Epoch [4/500], Accuracy: 75.14%, Loss: 0.447668969631195
tensor(0.4301, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 81.22%
Epoch [5/500], Accuracy: 81.22%, Loss: 0.430120140314102
tensor(0.3143, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 84.53%
Epoch [6/500], Accuracy: 84.53%, Loss: 0.314254701137543
tensor(0.4557, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 79.01%
Epoch [7/500], Accuracy: 79.01%, Loss: 0.455659210681915
tensor(0.2998, device='cuda:0', grad_fn=<NllLossBackward0>)
Ac


KeyboardInterrupt



In [10]:
for batch in train_loader:
    print(batch)
    batch.to(device)
    out = model(batch)
    out_single = model.GCN_single(batch)
    break

DataBatch(x=[798, 1], edge_index=[2, 2793], y=[128], edge_types=[2793, 1], graph_sizes=[128], batch=[798], ptr=[129])
